In [ ]:
'''
Tic-Tac-Toe Game dataset generation - Player 1 (AI) vs. Player 2 (Human)
'''

# Multiple agents can be selected:
# 1: Random agent
# 2: Rule-based agent (no mistakes)
# 3: Rule-based agent (with % of mistakes)
# 4: Random Forest agent

# Choose agent
agent = 1
p_mistake = 0.1  # % chance of making a mistake (only for agent 3)

import random

# Initialize the dataset
dataset = []

def print_board(board):
    """
    Print the current state of the board.
    """
    print("\n")
    for i in range(0, 9, 3):
        print(board[i], "|", board[i + 1], "|", board[i + 2])
        if i < 6:
            print("---------")
    print("\n")

def is_winner(board, player):
    """
    Check if the specified player has won.
    """
    winning_combinations = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
        [0, 4, 8], [2, 4, 6]              # Diagonals
    ]
    for combo in winning_combinations:
        if all(board[i] == player for i in combo):
            return True
    return False

def is_board_full(board):
    """
    Check if the board is full.
    """
    return all(cell != 0 for cell in board)

def get_empty_cells(board):
    """
    Get a list of indices of empty cells.
    """
    return [i for i, cell in enumerate(board) if cell == 0]


if agent == 1:

    def player_one_move(board):
        """
        Player 1 (X) makes a random move.
        """
        return random.choice(get_empty_cells(board))
    
if agent == 2:

    def player_one_move(board):
        """
        Player 1 (X) makes a move using a rule-based AI.
        """
        # Check if the AI can win in the next move
        for move in get_empty_cells(board):
            board[move] = 1  # Simulate the move
            if is_winner(board, 1):
                board[move] = 0  # Undo the move
                return move
            board[move] = 0  # Undo the move

        # Check if the opponent can win in the next move and block them
        for move in get_empty_cells(board):
            board[move] = 2  # Simulate the opponent's move
            if is_winner(board, 2):
                board[move] = 0  # Undo the move
                return move
            board[move] = 0  # Undo the move

        # Take the center if available
        if board[4] == 0:
            return 4

        # Take a corner if available
        corners = [0, 2, 6, 8]
        for move in corners:
            if board[move] == 0:
                return move

        # Take an edge if available
        edges = [1, 3, 5, 7]
        for move in edges:
            if board[move] == 0:
                return move

        # If no move is found (should not happen), return a random move
        return random.choice(get_empty_cells(board))
    
elif agent == 3:
    def player_one_move(board):
    """
    Player 1 (X) makes a move using a rule-based AI with occasional random mistakes.
    """
    # Introduce a chance of making a random mistake
    if random.random() < p_mistake:  # % chance
        return random.choice(get_empty_cells(board))

    # Check if the AI can win in the next move
    for move in get_empty_cells(board):
        board[move] = 1  # Simulate the move
        if is_winner(board, 1):
            board[move] = 0  # Undo the move
            return move
        board[move] = 0  # Undo the move

    # Check if the opponent can win in the next move and block them
    for move in get_empty_cells(board):
        board[move] = 2  # Simulate the opponent's move
        if is_winner(board, 2):
            board[move] = 0  # Undo the move
            return move
        board[move] = 0  # Undo the move

    # If it's the first move, choose a completely random empty cell
    if board.count(0) == 9:  # Check if the board is empty (first move)
        return random.choice(get_empty_cells(board))

    # Otherwise, prioritize corners, then center, then edges
    corners = [0, 2, 6, 8]
    empty_corners = [move for move in corners if board[move] == 0]
    if empty_corners:
        return random.choice(empty_corners)

    if board[4] == 0:  # Take the center if available
        return 4

    edges = [1, 3, 5, 7]
    empty_edges = [move for move in edges if board[move] == 0]
    if empty_edges:
        return random.choice(empty_edges)

    # If no move is found (should not happen), return a random move
    return random.choice(get_empty_cells(board))

elif agent == 4:
    def player_one_move(board):
    """
    Player 1 (X) makes a move using the Random Forest model.
    """
    model = joblib.load("tictactoe_rf_player1.pkl")
    return model.predict([board])[0]


def player_two_move(board):
    """
    Player 2 (O) makes a move based on user input.
    """
    while True:
        try:
            move = int(input("Enter your move (0-8): "))
            if move < 0 or move > 8:
                print("Invalid move! Please enter a number between 0 and 8.")
            elif board[move] != 0:
                print("Cell already occupied! Try again.")
            else:
                return move
        except ValueError:
            print("Invalid input! Please enter a number between 0 and 8.")

def play_game():
    """
    Play a game of Tic-Tac-Toe and record all board states and moves.
    """
    board = [0] * 9  # Reset the board

    print("New game! You are Player 2 (O). Player 1 (X) will play using a rule-based AI.")
    print_board(board)

    while True:
        # Player 1 (X) makes a move
        print("Player 1 (X) is making a move...")
        move_x = player_one_move(board)
        if move_x is not None:
            # Record the current board state and the move
            dataset.append((board.copy(), move_x))
            # Update the board
            board[move_x] = 1
            print_board(board)

            # Check if Player 1 (X) has won
            if is_winner(board, 1):
                print("Player 1 (X) wins!")
                break

            # Check if the board is full
            if is_board_full(board):
                print("It's a draw!")
                break

        # Player 2 (O) makes a move
        print("Your turn (Player 2 - O):")
        move_o = player_two_move(board)
        if move_o is not None:
            # Record the current board state and the move
            dataset.append((board.copy(), move_o))
            # Update the board
            board[move_o] = 2
            print_board(board)

            # Check if Player 2 (O) has won
            if is_winner(board, 2):
                print("Player 2 (O) wins!")
                break

            # Check if the board is full
            if is_board_full(board):
                print("It's a draw!")
                break

def save_dataset(filename):
    """
    Save the dataset to a file in the format: [board_state], move
    """
    with open(filename, "w") as f:
        for board, move in dataset:
            f.write(f"{board}, {move}\n")
    print(f"Dataset saved to {filename}")

# Main loop to play multiple games
while True:
    play_game()
    continue_game = input("Do you want to play another game? (y/n): ").strip().lower()
    if continue_game != 'y':
        break

# Save the dataset
save_dataset("tic_tac_toe_dataset__player2_temp.txt")

In [6]:
'''
Tic-Tac-Toe Game dataset generation - Player 1 (Human) vs. Player 2 (AI)
'''

# Multiple agents can be selected:
# 1: Random agent
# 2: Rule-based agent (no mistakes)
# 3: Rule-based agent (with % of mistakes)
# 4: Random Forest agent

# Choose agent
agent = 3
p_mistake = 0.7  # % chance of making a mistake (only for agent 3)

import random
import joblib

# Initialize the dataset
dataset = []

def print_board(board):
    """
    Print the current state of the board.
    """
    print("\n")
    for i in range(0, 9, 3):
        print(board[i], "|", board[i + 1], "|", board[i + 2])
        if i < 6:
            print("---------")
    print("\n")

def is_winner(board, player):
    """
    Check if the specified player has won.
    """
    winning_combinations = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Rows
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Columns
        [0, 4, 8], [2, 4, 6]              # Diagonals
    ]
    for combo in winning_combinations:
        if all(board[i] == player for i in combo):
            return True
    return False

def is_board_full(board):
    """
    Check if the board is full.
    """
    return all(cell != 0 for cell in board)

def get_empty_cells(board):
    """
    Get a list of indices of empty cells.
    """
    return [i for i, cell in enumerate(board) if cell == 0]


if agent == 1:

    def player_two_move(board):
        """
        Player 2 (O) makes a random move.
        """
        return random.choice(get_empty_cells(board))
    
if agent == 2:

    def player_two_move(board):
        """
        Player 2 (O) makes a move using a rule-based AI.
        """
        # Check if the AI can win in the next move
        for move in get_empty_cells(board):
            board[move] = 2  # Simulate the move
            if is_winner(board, 2):
                board[move] = 0  # Undo the move
                return move
            board[move] = 0  # Undo the move

        # Check if the opponent can win in the next move and block them
        for move in get_empty_cells(board):
            board[move] = 1  # Simulate the opponent's move
            if is_winner(board, 1):
                board[move] = 0  # Undo the move
                return move
            board[move] = 0  # Undo the move

        # Take the center if available
        if board[4] == 0:
            return 4

        # Take a corner if available
        corners = [0, 2, 6, 8]
        for move in corners:
            if board[move] == 0:
                return move

        # Take an edge if available
        edges = [1, 3, 5, 7]
        for move in edges:
            if board[move] == 0:
                return move

        # If no move is found (should not happen), return a random move
        return random.choice(get_empty_cells(board))
    
elif agent == 3:
    def player_two_move(board):
        """
        Player 2 (O) makes a move using a rule-based AI with occasional random mistakes.
        """
        # Introduce a chance of making a random mistake
        if random.random() < p_mistake:  # % chance
            return random.choice(get_empty_cells(board))

        # Check if the AI can win in the next move
        for move in get_empty_cells(board):
            board[move] = 2  # Simulate the move
            if is_winner(board, 2):
                board[move] = 0  # Undo the move
                return move
            board[move] = 0  # Undo the move

        # Check if the opponent can win in the next move and block them
        for move in get_empty_cells(board):
            board[move] = 1  # Simulate the opponent's move
            if is_winner(board, 1):
                board[move] = 0  # Undo the move
                return move
            board[move] = 0  # Undo the move

        # If it's the first move, choose a completely random empty cell
        if board.count(0) == 9:  # Check if the board is empty (first move)
            return random.choice(get_empty_cells(board))

        # Otherwise, prioritize corners, then center, then edges
        corners = [0, 2, 6, 8]
        empty_corners = [move for move in corners if board[move] == 0]
        if empty_corners:
            return random.choice(empty_corners)

        if board[4] == 0:  # Take the center if available
            return 4

        edges = [1, 3, 5, 7]
        empty_edges = [move for move in edges if board[move] == 0]
        if empty_edges:
            return random.choice(empty_edges)

        # If no move is found (should not happen), return a random move
        return random.choice(get_empty_cells(board))

elif agent == 4:
    def player_two_move(board):
        """
        Player 2 (O) makes a move using the Random Forest model.
        """
        model = joblib.load("tictactoe_rf_player2.pkl")
        return model.predict([board])[0]


def player_one_move(board):
    """
    Player 1 (X) makes a move based on user input.
    """
    while True:
        try:
            move = int(input("Enter your move (0-8): "))
            if move < 0 or move > 8:
                print("Invalid move! Please enter a number between 0 and 8.")
            elif board[move] != 0:
                print("Cell already occupied! Try again.")
            else:
                return move
        except ValueError:
            print("Invalid input! Please enter a number between 0 and 8.")

def play_game():
    """
    Play a game of Tic-Tac-Toe and record all board states and moves.
    """
    board = [0] * 9  # Reset the board

    print("New game! You are Player 1 (X). Player 2 (O) will play using a rule-based AI.")
    print_board(board)

    while True:
        # Player 1 (X) makes a move
        print("Your turn (Player 1 - X):")
        move_x = player_one_move(board)
        if move_x is not None:
            # Record the current board state and the move
            dataset.append((board.copy(), move_x))
            # Update the board
            board[move_x] = 1
            print_board(board)

            # Check if Player 1 (X) has won
            if is_winner(board, 1):
                print("Player 1 (X) wins!")
                break

            # Check if the board is full
            if is_board_full(board):
                print("It's a draw!")
                break

        # Player 2 (O) makes a move
        print("Player 2 (O) is making a move...")
        move_o = player_two_move(board)
        if move_o is not None:
            # Record the current board state and the move
            dataset.append((board.copy(), move_o))
            # Update the board
            board[move_o] = 2
            print_board(board)

            # Check if Player 2 (O) has won
            if is_winner(board, 2):
                print("Player 2 (O) wins!")
                break

            # Check if the board is full
            if is_board_full(board):
                print("It's a draw!")
                break

def save_dataset(filename):
    """
    Save the dataset to a file in the format: [board_state], move
    """
    with open(filename, "w") as f:
        for board, move in dataset:
            f.write(f"{board}, {move}\n")
    print(f"Dataset saved to {filename}")

# Main loop to play multiple games
while True:
    play_game()
    continue_game = input("Do you want to play another game? (y/n): ").strip().lower()
    if continue_game != 'y':
        break

# Save the dataset
save_dataset("tic_tac_toe_dataset__player1_temp.txt")

New game! You are Player 1 (X). Player 2 (O) will play using a rule-based AI.


0 | 0 | 0
---------
0 | 0 | 0
---------
0 | 0 | 0


Your turn (Player 1 - X):


0 | 0 | 0
---------
0 | 1 | 0
---------
0 | 0 | 0


Player 2 (O) is making a move...


0 | 2 | 0
---------
0 | 1 | 0
---------
0 | 0 | 0


Your turn (Player 1 - X):


1 | 2 | 0
---------
0 | 1 | 0
---------
0 | 0 | 0


Player 2 (O) is making a move...


1 | 2 | 0
---------
0 | 1 | 0
---------
2 | 0 | 0


Your turn (Player 1 - X):


1 | 2 | 0
---------
0 | 1 | 0
---------
2 | 0 | 1


Player 1 (X) wins!
New game! You are Player 1 (X). Player 2 (O) will play using a rule-based AI.


0 | 0 | 0
---------
0 | 0 | 0
---------
0 | 0 | 0


Your turn (Player 1 - X):


0 | 0 | 0
---------
0 | 1 | 0
---------
0 | 0 | 0


Player 2 (O) is making a move...


2 | 0 | 0
---------
0 | 1 | 0
---------
0 | 0 | 0


Your turn (Player 1 - X):


2 | 0 | 1
---------
0 | 1 | 0
---------
0 | 0 | 0


Player 2 (O) is making a move...


2 | 0 | 1
---------
0